In [1]:
#Utility
import urllib
import os
import datetime

#Data Science
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm

#Google API
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta2

## Mobility

In [20]:
#Import mobility from url
url = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'
mobility = pd.read_csv(url)

#Transform mobility to time series
#startdate = datetime.datetime.strptime('2020-02-15','%Y-%m-%d').date()
#enddate = datetime.datetime.strptime('2020-04-11','%Y-%m-%d').date()

filterlist = ['retail_and_recreation_percent_change_from_baseline',
'grocery_and_pharmacy_percent_change_from_baseline',
'parks_percent_change_from_baseline',
'transit_stations_percent_change_from_baseline',
'workplaces_percent_change_from_baseline',
'residential_percent_change_from_baseline']

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
#Get ONLY US mobility
country = mobility['country_region'].value_counts()
countrylist = country.index.to_list()
usmobility = mobility[mobility['country_region']=='United States']

#Change column names
usmobility = usmobility.rename(columns={"sub_region_1":"Province_State", "sub_region_2": "Admin2"})

#Split "county" text out of county
countylist = []
txt = usmobility['Admin2'].to_list()
for i in txt:
    i = str(i)
    countylist.append(i.replace(' County', ''))
usmobility['Admin2'] = countylist

usmobility['Lookup'] = usmobility['Admin2'] + usmobility['Province_State'] 
mobilitylist = usmobility['Lookup'].value_counts().index.to_list()

### WARNING!!! For the code block below!

In [54]:
#Loop to forward fill missing mobility !!!!!!!!!WARNING!THIS TAKES A WHILE TO RUN!!!!!!
for i in mobilitylist:
    usmobility[usmobility['Lookup']==i] = usmobility[usmobility['Lookup']==i].fillna(method='ffill')

In [56]:
usmobility['Lookup'] = usmobility['Lookup'] + usmobility['date']
lookuplist = usmobility['Lookup'].to_list()
usmobility = usmobility.transpose()
usmobility.columns = lookuplist
usmobility = usmobility.transpose()

filterlist1 = filterlist.copy()
filterlist1.append('Lookup')
usmobility = usmobility.loc[:,filterlist1]

## Cases from John Hopkins

In [58]:
#Import confirmed, deaths and recovered cases from url
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
#recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

In [59]:
#Formating confirmed as df ready for import for maindf
df = confirmed.copy()
xlist = ['UID', 'iso2', 'iso3', 'code3','Country_Region', 'Lat', 'Long_','Combined_Key']

df['Lookup'] = df['Admin2'] + df['Province_State']
includelist = [x for x in df.columns if x not in xlist]
datelist = includelist.copy()
datelist.pop(0)
datelist.pop(0)
datelist.remove('Lookup')
df = df.loc[:,includelist]

#Transpose and change index
lookuplist = df['Lookup'].to_list()
df = df.transpose()
df.columns = lookuplist
df = df.transpose()

In [60]:
#Formating deaths as df1 ready for import for maindf
df1 = deaths.copy()
xlist = ['UID', 'iso2', 'iso3', 'code3','Country_Region', 'Lat', 'Long_','Combined_Key']

df1['Lookup'] = df1['Admin2'] + df1['Province_State']
includelist = [x for x in df1.columns if x not in xlist]
datelist = includelist.copy()
datelist.pop(0)
datelist.pop(0)
datelist.pop(0)
datelist.pop(0)
datelist.remove('Lookup')
df1 = df1.loc[:,includelist]

#Transpose and change index
lookuplist = df1['Lookup'].to_list()
df1 = df1.transpose()
df1.columns = lookuplist
df1 = df1.transpose()

#Create a lookup list to loop over for maindf
lookuplist = df1['Lookup'].value_counts().index.to_list()
lookuplist = sorted(lookuplist)

## Contructing Main Dataframe

In [61]:
#Construction of maindf
colnames = ['Lookup','State','County','FIPS','Population','Date','Confirmed','Deaths']
#'retail_and_recreation_percent_change_from_baseline',
#'grocery_and_pharmacy_percent_change_from_baseline',
#'parks_percent_change_from_baseline',
#'transit_stations_percent_change_from_baseline',
#'workplaces_percent_change_from_baseline',
#'residential_percent_change_from_baseline']

maindf = pd.DataFrame(index=range(0,len(lookuplist)*len(datelist)), columns=colnames)

#Start loop for confirmed
j = 0
for i in lookuplist:
    testdf = df.loc[i,:]
    testdf1 = df1.loc[i,:]
    
    #Confirmed cases
    timeseries = testdf[3:-1]
    date = timeseries.index.to_list()
    timeseries = timeseries.to_list()
    length = len(timeseries)
    
    #Deaths
    timeseries1 = testdf1[4:-1].to_list()
    
    maindf.iloc[j:(j+length),colnames.index('Lookup')] = i
    maindf.iloc[j:(j+length),colnames.index('State')] = testdf[2]
    maindf.iloc[j:(j+length),colnames.index('County')] = testdf[1]
    maindf.iloc[j:(j+length),colnames.index('FIPS')] = testdf[0]
    maindf.iloc[j:(j+length),colnames.index('Population')] = testdf1[3]
    maindf.iloc[j:(j+length),colnames.index('Date')] = date
    maindf.iloc[j:(j+length),colnames.index('Confirmed')] = timeseries
    maindf.iloc[j:(j+length),colnames.index('Deaths')] = timeseries1
    j = j+length
    
maindf.loc[:,'Date1'] = pd.to_datetime(maindf['Date'],format='%m/%d/%y')
#test = maindf['Date1'].astype(str).tolist()
maindf['Lookup'] = maindf['Lookup'] + maindf['Date1'].astype(str).tolist()

labels = maindf['Lookup'].to_list()
maindf = maindf.transpose()
maindf.columns = labels
maindf = maindf.transpose()

In [62]:
#Merge maindf and mobility to finaldf
finaldf = maindf.merge(usmobility, left_on='Lookup', right_on='Lookup')

In [63]:
finaldf

,Lookup,State,County,FIPS,Population,Date,Confirmed,Deaths,Date1,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AbbevilleSouth Carolina2020-02-15,South Carolina,Abbeville,45001,24527,2/15/20,0,0,2020-02-15,14,5,NaN,NaN,1,NaN
1,AbbevilleSouth Carolina2020-02-16,South Carolina,Abbeville,45001,24527,2/16/20,0,0,2020-02-16,14,-8,NaN,NaN,0,NaN
2,AbbevilleSouth Carolina2020-02-17,South Carolina,Abbeville,45001,24527,2/17/20,0,0,2020-02-17,4,-12,NaN,NaN,-15,NaN
3,AbbevilleSouth Carolina2020-02-18,South Carolina,Abbeville,45001,24527,2/18/20,0,0,2020-02-18,-2,0,NaN,NaN,1,NaN
4,AbbevilleSouth Carolina2020-02-19,South Carolina,Abbeville,45001,24527,2/19/20,0,0,2020-02-19,9,-6,NaN,NaN,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148474,ZavalaTexas2020-04-06,Texas,Zavala,48507,11840,4/6/20,0,0,2020-04-06,-51,-16,NaN,NaN,-34,NaN
148475,ZavalaTexas2020-04-07,Texas,Zavala,48507,11840,4/7/20,0,0,2020-04-07,-51,-16,NaN,NaN,-40,NaN
148476,ZavalaTexas2020-04-08,Texas,Zavala,48507,11840,4/8/20,0,0,2020-04-08,-51,-16,NaN,NaN,-39,NaN
148477,ZavalaTexas2020-04-09,Texas,Zavala,48507,11840,4/9/20,0,0,2020-04-09,-51,-16,NaN,NaN,-32,NaN
